In [ ]:
def limpiar_dataframe_inicial(df: pd.DataFrame) -> pd.DataFrame:
    """
    Realiza una limpieza general inicial en el DataFrame.

    Esta función toma un DataFrame, elimina los espacios en blanco
    de las columnas de tipo 'object' (texto), convierte la columna 'date'
    a formato datetime, la ordena cronológicamente, elimina filas con
    fechas nulas y extrae la fecha y la hora en nuevas columnas.

    Args:
        df (pd.DataFrame): El DataFrame de entrada que se va a limpiar.

    Returns:
        pd.DataFrame: El DataFrame procesado y limpio.
    """
    df_copia = df.copy()

    for col in df_copia.columns:
        if df_copia[col].dtype == 'object':
            df_copia[col] = df_copia[col].str.strip()

    df_copia['date'] = pd.to_datetime(df_copia['date'], dayfirst=True)
    df_copia = df_copia.sort_values(by='date', ascending=True)
    df_copia = df_copia.dropna(subset=['date'])

    #df_copia['fecha'] = df_copia['date'].dt.date
    #df_copia['hora'] = df_copia['date'].dt.time
    #df_copia = df_copia.replace(['nan', 'NaN', 'None', 'NULL', 'null'], np.nan)


    return df_copia

def limpiar_columnas_categoricas(df: pd.DataFrame) -> pd.DataFrame:
    """
    Limpia y estandariza las columnas categóricas del DataFrame.

    Esta función corrige valores mal escritos y maneja los datos nulos
    en las columnas 'WeekStatus', 'Day_of_week' y 'Load_Type'.
    Los valores incorrectos se reemplazan por su versión correcta y los
    nulos se rellenan utilizando el método 'forward fill'.

    Args:
        df (pd.DataFrame): DataFrame con las columnas categóricas a limpiar.

    Returns:
        pd.DataFrame: DataFrame con las columnas categóricas estandarizadas.
    """
    df_copia = df.copy()

    # Mapeo de correcciones para cada columna
    mapa_correcciones = {
        'WeekStatus': {
            'wEEKDAY': 'Weekday', 'wEEKEND': 'Weekend', 'NAN': np.nan
        },
        'Day_of_week': {
            'mONDAY': 'Monday', 'tUESDAY': 'Tuesday', 'wEDNESDAY': 'Wednesday',
            'tHURSDAY': 'Thursday', 'fRIDAY': 'Friday', 'sATURDAY': 'Saturday',
            'sUNDAY': 'Sunday', 'NAN': np.nan
        },
        'Load_Type': {
            'lIGHT_lOAD': 'Light_Load', 'mEDIUM_lOAD': 'Medium_Load',
            'mAXIMUM_lOAD': 'Maximum_Load', 'NAN': np.nan
        }
    }

    columnas_categoricas = ['WeekStatus', 'Day_of_week', 'Load_Type']

    for col in columnas_categoricas:
        df_copia[col] = df_copia[col].replace(mapa_correcciones[col])
        df_copia[col] = df_copia[col].astype(str)

    return df_copia


def limpiar_columnas_numericas(df: pd.DataFrame) -> pd.DataFrame:
    """
    Limpia y estandariza un conjunto predefinido de columnas numéricas.

    La función identifica valores no numéricos en columnas que deberían serlo,
    los convierte a NaN (Not a Number) y luego utiliza el método 'forward fill'
    para imputar los valores faltantes. Finalmente, asegura que todas las
    columnas procesadas tengan el tipo de dato 'float'.

    Args:
        df (pd.DataFrame): DataFrame con columnas numéricas a procesar.

    Returns:
        pd.DataFrame: DataFrame con las columnas numéricas limpias y
                      en formato float.
    """
    df_copia = df.copy()

    columnas_numericas = [
        'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh',
        'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)',
        'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor',
        'NSM', 'mixed_type_col'
    ]

    for col in columnas_numericas:
        # Forzar valores no numéricos a NaN
        df_copia[col] = pd.to_numeric(df_copia[col], errors='coerce')

        # Asegurar el tipo float
        df_copia[col] = df_copia[col].astype(float)

    return df_copia


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler


class SteelEnergyEDA:
    """
    Clase para realizar análisis exploratorio de datos (EDA), limpieza, visualización y preprocesamiento
    de un dataset de energía y acero.

    Esta clase carga datos desde un archivo CSV, realiza limpieza inicial, genera visualizaciones,
    aplica transformaciones y guarda los datos procesados.

    Attributes:
        file_path (str): Ruta del archivo CSV que contiene el dataset.
        df (pd.DataFrame): DataFrame cargado desde el archivo.
        num_cols (list): Lista de nombres de columnas numéricas.
        cat_cols (list): Lista de nombres de columnas categóricas.
        scaler (StandardScaler): Objeto de escalado para normalizar datos numéricos.
        preprocessor (ColumnTransformer): Transformer usado en preprocesamiento.
    """
    def __init__(self, file_path):
        self.file_path = file_path
        self.df = None
        self.num_cols = []
        self.cat_cols = []
        self.scaler = None
        self.preprocessor = None

    # Cargar Datos
    def load_data(self):
        """
        Carga el dataset desde un archivo CSV en el atributo `df`.

        Returns:
            pd.DataFrame: Primeras filas del dataset cargado.
        """
        self.df = pd.read_csv(self.file_path)
        print(f"Se cargo el dataset correctamente: {self.df.shape}")

        return self.df.head()

    # Resumen Básico
    def basic_summary(self):
        print("\n--- INFO ---")
        self.df.info()
        print("\n--- STATS ---")
        display(self.df.describe())


    def visualize(self):
        """
        Genera visualizaciones para el dataset.

        Incluye:
            - Heatmap de correlación para variables numéricas.
            - Histogramas para cada variable numérica.
            - Pairplot para correlaciones.
            - Gráficos de barras para variables categóricas.
        """
        sns.set(style="whitegrid", palette="muted", font_scale=1.1)

        # 1. HeatMap Correlación
        if len(self.num_cols) > 1:
            plt.figure(figsize=(10, 8))
            corr = self.df[self.num_cols].corr()
            sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
            plt.title("Correlation Heatmap", fontsize=16)
            plt.show()

        # 2. Histogramas Variables Numéricas
        for col in self.num_cols:
            plt.figure(figsize=(6, 4))
            sns.histplot(self.df[col], kde=True, color="skyblue", edgecolor="black")
            plt.title(f"Distribution of {col}", fontsize=14)
            plt.xlabel(col)
            plt.ylabel("Frequency")
            plt.show()

        # 3. Pairplot para correlaciones de variables numéricas
        if len(self.num_cols) > 1:
            sns.pairplot(self.df[self.num_cols], corner=True, diag_kind="kde")
            plt.suptitle("Pairplot of Numeric Variables", y=1.02, fontsize=16)
            plt.show()

        # 4. Bar charts para variables categóricas
        for col in self.cat_cols:
            plt.figure(figsize=(8, 5))
            sns.countplot(x=col, data=self.df, order=self.df[col].value_counts().index, palette="pastel")
            plt.title(f"Count of {col}", fontsize=14)
            plt.xlabel(col)
            plt.ylabel("Count")
            plt.xticks(rotation=45, ha="right")
            plt.tight_layout()
            plt.show()


    def clean_data(self):
        """
        Realiza limpieza del dataset.

        Pasos:
            - Limpieza general usando funciones externas.
            - Reemplazo de valores nulos estándar.
            - Identificación de columnas numéricas y categóricas.
            - Eliminación de duplicados.
            - Imputación de valores faltantes.
            - Eliminación de outliers usando método IQR.

        Returns:
            pd.DataFrame: Primeras filas del dataset limpio.
        """
        print("\n--- LIMPIEZA ---")

        initial_shape = self.df.shape

        ## Limpieza General
        self.df = limpiar_dataframe_inicial(self.df)
        self.df = limpiar_columnas_categoricas(self.df)
        self.df = limpiar_columnas_numericas(self.df)
        self.df = self.df.replace(['nan', 'NaN', 'None', 'NULL', 'null'], np.nan)


        self.num_cols = self.df.select_dtypes(include=np.number).columns
        self.cat_cols = self.df.select_dtypes(include=['object', 'category']).columns

        # Eliminar duplicados
        self.df = self.df.drop_duplicates()

        # Valores Faltantes
        self.df[self.num_cols] = self.df[self.num_cols].fillna(self.df[self.num_cols].median())
        for col in self.cat_cols:
            self.df[col] = self.df[col].fillna(self.df[col].mode()[0])

        # Quitar outliers (IQR)
        for col in self.num_cols:
            Q1 = self.df[col].quantile(0.25)
            Q3 = self.df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
            self.df = self.df[(self.df[col] >= lower) & (self.df[col] <= upper)]

        print(f"Resultados de Limpieza de datos: {initial_shape[0]} → {self.df.shape[0]} rows")
        return self.df.head()

    def preprocess_data(self):
        """
        Realiza preprocesamiento del dataset.

        Incluye:
            - Extracción de características temporales.
            - Encoding cíclico para variables de fecha.
            - Normalización de características numéricas.
            - Encoding de variables categóricas ordinales.
            - Creación de un DataFrame procesado listo para análisis o modelos.

        Returns:
            pd.DataFrame: Dataset procesado.
        """
        print("\n--- PREPROCESAMIENTO ---")

        print(self.cat_cols)
        print(self.num_cols)

        # Extraer features de tiempo
        self.df['date_hour'] = self.df["date"].dt.hour
        self.df['date_day'] = self.df["date"].dt.day
        self.df['date_month'] = self.df["date"].dt.month
        self.df['date_year'] = self.df["date"].dt.year

        # Encoding ciclico
        for col, max_val in [('date_hour', 24), ('date_month', 12), ('date_day', 31)]:
            self.df[f'{col}_sin'] = np.sin(2 * np.pi * self.df[col] / max_val)
            self.df[f'{col}_cos'] = np.cos(2 * np.pi * self.df[col] / max_val)



        # Dividir Features (Usage_kWh se normaliza junto con las demás)
        numeric_features = [
            'Usage_kWh', 'CO2(tCO2)', 'NSM',
            'Leading_Current_Reactive_Power_kVarh',
            'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor',
            'date_hour_sin', 'date_hour_cos',
            'date_day_sin', 'date_day_cos',
            'date_month_sin', 'date_month_cos',
            'date_year'
        ]
        numeric_features = [col for col in numeric_features if col in self.df.columns]

        weekstatus_feature = ['WeekStatus'] if 'WeekStatus' in self.df.columns else []
        ordinal_features = ['Load_Type'] if 'Load_Type' in self.df.columns else []

        # Pipelines
        numeric_transformer = Pipeline(steps=[
            ('scaler', StandardScaler())
        ])

        weekstatus_transformer = Pipeline(steps=[
            ('label', OrdinalEncoder())
        ])

        ordinal_transformer = Pipeline(steps=[
            ('ordinal', OrdinalEncoder(categories=[["Light_Load", "Medium_Load", "Maximum_Load"]]))
        ])

        # Transformer
        self.preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('week', weekstatus_transformer, weekstatus_feature),
                ('ord', ordinal_transformer, ordinal_features)
            ],
            sparse_threshold=0

        )

        self.df_processed = self.preprocessor.fit_transform(self.df)

        processed_cols = numeric_features


        if weekstatus_feature:
            processed_cols += weekstatus_feature

        if ordinal_features:
            processed_cols += ordinal_features

        print("Shape of processed data:", self.df_processed.shape)
        print("Number of processed columns:", len(processed_cols))

        self.df_processed = pd.DataFrame(self.df_processed, columns=processed_cols)
        self.save_data(self.df_processed)

        return self.df_processed


    def save_data(self, df, output_path="cleaned_steel_energy_v2.csv"):
        """
        Guarda el DataFrame procesado en un archivo CSV.

        Args:
            df (pd.DataFrame): Dataset a guardar.
            output_path (str): Ruta del archivo de salida (default: "cleaned_steel_energy_v2.csv").
        """
        df.to_csv(output_path, index=False)
        print(f"Saved cleaned dataset to '{output_path}'")


In [ ]:
# Pipeline
eda = SteelEnergyEDA("../../data/raw/steel_energy_modified.csv")

# 1. Cargar Datos
eda.load_data()

# 2. Limpiar Datos
eda.clean_data()

# 3. Resumen
eda.basic_summary()

# 4. Visualizaciones
eda.visualize()

# 5. Preprocessamiento
output = eda.preprocess_data()

output.head()
output.describe()


In [ ]:
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split


class SteelEnergyModeling:
    """
    Clase para entrenar modelos de machine learning sobre datos procesados.

    Attributes:
        df_processed (pd.DataFrame): DataFrame con los datos ya procesados por SteelEnergyEDA.
        target_col (str): Nombre de la columna objetivo a predecir.
        preprocessor: ColumnTransformer usado para normalizar datos.
        X_train (pd.DataFrame): Features de entrenamiento.
        X_val (pd.DataFrame): Features de validación.
        X_test (pd.DataFrame): Features de prueba.
        y_train (pd.Series): Target de entrenamiento.
        y_val (pd.Series): Target de validación.
        y_test (pd.Series): Target de prueba.
        models (dict): Diccionario con los modelos entrenados {'nombre_modelo': modelo}.
    """

    def __init__(self, df_processed, preprocessor=None, target_col='Usage_kWh'):
        """
        Inicializa el objeto SteelEnergyModeling.

        Args:
            df_processed (pd.DataFrame): DataFrame procesado que contiene features y target.
            preprocessor: ColumnTransformer usado en preprocesamiento (para des-normalizar).
            target_col (str): Nombre de la columna objetivo. Default: 'Usage_kWh'.
        """
        self.df_processed = df_processed
        self.preprocessor = preprocessor
        self.target_col = target_col
        self.target_scaler = None
        self.X_train = None
        self.X_val = None
        self.X_test = None
        self.y_train = None
        self.y_val = None
        self.y_test = None
        self.models = {}

        # Obtener el scaler del target desde el ColumnTransformer
        if preprocessor is not None:
            # El scaler está en el transformer 'num'
            self.target_scaler = preprocessor.named_transformers_['num'].named_steps['scaler']

    def _inverse_transform_target(self, y_scaled):
        """
        Des-normaliza los valores del target.

        Args:
            y_scaled: Valores normalizados del target (array o series).

        Returns:
            Valores des-normalizados.
        """
        if self.target_scaler is None:
            return y_scaled

        # Asegurar que es un array 2D
        y_scaled_2d = np.array(y_scaled).reshape(-1, 1)

        # El StandardScaler normalizó todas las columnas numéricas juntas
        # Necesitamos usar los parámetros específicos del target (primera columna = Usage_kWh)
        target_mean = self.target_scaler.mean_[0]
        target_scale = self.target_scaler.scale_[0]

        # Des-normalizar: y = y_scaled * scale + mean
        y_original = y_scaled_2d * target_scale + target_mean

        return y_original.flatten()

    def prepare_data(self, test_size=0.2, val_size=0.2, random_state=42):
        """
        Divide los datos en conjuntos de entrenamiento, validación y prueba.

        Args:
            test_size (float): Proporción del conjunto de prueba (0-1). Default: 0.2.
            val_size (float): Proporción del conjunto de validación (0-1). Default: 0.2.
            random_state (int): Semilla para reproducibilidad. Default: 42.

        Returns:
            None. Actualiza los atributos X_train, X_val, X_test, y_train, y_val, y_test del objeto.
        """
        X = self.df_processed.drop(columns=[self.target_col])
        y = self.df_processed[self.target_col]

        # Primero dividir en train+val y test
        X_temp, self.X_test, y_temp, self.y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state
        )

        # Luego dividir train+val en train y val
        val_size_adjusted = val_size / (1 - test_size)
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            X_temp, y_temp, test_size=val_size_adjusted, random_state=random_state
        )

        print(f"Train: {self.X_train.shape}, Val: {self.X_val.shape}, Test: {self.X_test.shape}")

    def train_linear_regression(self):
        """
        Entrena un modelo de regresión lineal.

        Args:
            None.

        Returns:
            None. Guarda el modelo entrenado en self.models['linear_regression']
            e imprime el R² en el conjunto de validación.
        """
        lr = LinearRegression()
        lr.fit(self.X_train, self.y_train)
        self.models['linear_regression'] = lr

        y_pred = lr.predict(self.X_val)
        print(f"Linear Regression - Val R²: {r2_score(self.y_val, y_pred):.4f}")

    def train_xgboost(self, **params):
        """
        Entrena un modelo XGBoost.

        Args:
            **params: Parámetros opcionales para XGBRegressor (ej: n_estimators=100,
                     max_depth=6, learning_rate=0.1, random_state=42).

        Returns:
            None. Guarda el modelo entrenado en self.models['xgboost']
            e imprime el R² en el conjunto de validación.
        """
        xgb_model = xgb.XGBRegressor(**params)
        xgb_model.fit(self.X_train, self.y_train)
        self.models['xgboost'] = xgb_model

        y_pred = xgb_model.predict(self.X_val)
        print(f"XGBoost - Val R²: {r2_score(self.y_val, y_pred):.4f}")

    def evaluate_models(self, model_names=None):
        """
        Evalúa los modelos entrenados en los conjuntos de validación y prueba.
        Des-normaliza las predicciones antes de calcular métricas.

        Args:
            model_names (list): Lista de nombres de modelos a evaluar.
                               Si es None, evalúa todos los modelos. Default: None.

        Returns:
            pd.DataFrame: DataFrame con las métricas (R², RMSE, MAE, CV) de validación y test.
                         CV = Coefficient of Variation = RMSE / mean(y_true)
        """
        if model_names is None:
            model_names = list(self.models.keys())

        results = []
        for name in model_names:
            if name not in self.models:
                print(f"Modelo '{name}' no encontrado. Saltando...")
                continue

            model = self.models[name]

            # Predicciones en validación (escaladas)
            y_val_pred_scaled = model.predict(self.X_val)

            # Des-normalizar
            y_val_true = self._inverse_transform_target(self.y_val.values)
            y_val_pred = self._inverse_transform_target(y_val_pred_scaled)

            # Métricas en validación
            val_r2 = r2_score(y_val_true, y_val_pred)
            val_rmse = np.sqrt(mean_squared_error(y_val_true, y_val_pred))
            val_mae = mean_absolute_error(y_val_true, y_val_pred)
            val_cv = val_rmse / np.abs(y_val_true.mean())

            # Predicciones en test (escaladas)
            y_test_pred_scaled = model.predict(self.X_test)

            # Des-normalizar
            y_test_true = self._inverse_transform_target(self.y_test.values)
            y_test_pred = self._inverse_transform_target(y_test_pred_scaled)

            # Métricas en test
            test_r2 = r2_score(y_test_true, y_test_pred)
            test_rmse = np.sqrt(mean_squared_error(y_test_true, y_test_pred))
            test_mae = mean_absolute_error(y_test_true, y_test_pred)
            test_cv = test_rmse / np.abs(y_test_true.mean())

            results.append({
                'model': name,
                'val_r2': val_r2,
                'val_rmse': val_rmse,
                'val_mae': val_mae,
                'val_cv': val_cv,
                'test_r2': test_r2,
                'test_rmse': test_rmse,
                'test_mae': test_mae,
                'test_cv': test_cv
            })

            print(f"{name}:")
            print(f"  Val  - R²: {val_r2:.4f}, RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}, CV: {val_cv:.4f}")
            print(f"  Test - R²: {test_r2:.4f}, RMSE: {test_rmse:.4f}, MAE: {test_mae:.4f}, CV: {test_cv:.4f}")

        return pd.DataFrame(results)

In [ ]:
# Crear objeto de modelado con los datos procesados y el preprocessor
modeling = SteelEnergyModeling(output, preprocessor=eda.preprocessor, target_col='Usage_kWh')

# Preparar datos (train/val/test split)
modeling.prepare_data(test_size=0.2, val_size=0.2, random_state=42)

# Entrenar regresión lineal
modeling.train_linear_regression()

# Entrenar XGBoost
modeling.train_xgboost(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)

# Evaluar modelos (des-normaliza automáticamente)
results = modeling.evaluate_models()
results